# Types Spec - Framework-Agnostic Field Specification

The `types.spec` module provides framework-agnostic field specifications for type validation and schema definition:

**Core Components:**
- **Spec**: Framework-agnostic field spec combining base_type + metadata
- **CommonMeta**: Standard metadata keys (NAME, NULLABLE, LISTABLE, VALIDATOR, DEFAULT, DEFAULT_FACTORY)
- **Meta**: Immutable key-value pairs for extensible metadata

**Key Features:**
- Type annotation generation (plain and Annotated)
- Nullable/listable modifiers
- Default values and factory functions (sync/async)
- Validator integration
- Functional updates (immutable API)
- Thread-safe annotated type caching
- Integration with Operable for structured outputs

In [1]:
from lionpride.types import CommonMeta, Meta, Spec, Undefined

## 1. CommonMeta - Standard Metadata Keys

CommonMeta defines standard metadata keys used throughout lionpride for field specifications.

In [2]:
# CommonMeta enum provides standard keys
print("Standard metadata keys:")
for meta in CommonMeta:
    print(f"  {meta.name} = {meta.value}")

# Access as enum members
print(f"\nNAME key: {CommonMeta.NAME.value}")
print(f"NULLABLE key: {CommonMeta.NULLABLE.value}")
print(f"DEFAULT key: {CommonMeta.DEFAULT.value}")

Standard metadata keys:
  NAME = name
  NULLABLE = nullable
  LISTABLE = listable
  VALIDATOR = validator
  DEFAULT = default
  DEFAULT_FACTORY = default_factory

NAME key: name
NULLABLE key: nullable
DEFAULT key: default


In [3]:
# CommonMeta.prepare() validates metadata constraints
valid_meta = CommonMeta.prepare(name="user_id", nullable=True, default="default_value")
print(f"Valid metadata: {valid_meta}")

# Cannot have both default and default_factory
try:
    invalid = CommonMeta.prepare(default="value", default_factory=lambda: "factory_value")
except ExceptionGroup as eg:
    print(f"\n✓ Validation error: {eg.exceptions[0]}")

Valid metadata: (Meta(key='name', value='user_id'), Meta(key='nullable', value=True), Meta(key='default', value='default_value'))

✓ Validation error: Cannot provide both 'default' and 'default_factory'


## 2. Basic Spec Construction

Spec combines a base type with metadata to define field specifications.

In [4]:
# Simple spec with just a type
basic_spec = Spec(str)
print(f"Basic spec: {basic_spec}")
print(f"Base type: {basic_spec.base_type}")
print(f"Metadata: {basic_spec.metadata}")

Basic spec: Spec(base_type=<class 'str'>, metadata=())
Base type: <class 'str'>
Metadata: ()


In [5]:
# Spec with metadata via kwargs
named_spec = Spec(int, name="user_id", nullable=False)
print(f"Named spec: {named_spec}")
print(f"Metadata count: {len(named_spec.metadata)}")
for meta in named_spec.metadata:
    print(f"  {meta.key} = {meta.value}")

Named spec: Spec(base_type=<class 'int'>, metadata=(Meta(key='name', value='user_id'), Meta(key='nullable', value=False)))
Metadata count: 2
  name = user_id
  nullable = False


In [6]:
# Spec with Meta objects as args
meta_spec = Spec(
    str, Meta("name", "email"), Meta("nullable", True), Meta("description", "User email address")
)
print(f"Spec with Meta objects: {meta_spec.metadata}")

Spec with Meta objects: (Meta(key='name', value='email'), Meta(key='nullable', value=True), Meta(key='description', value='User email address'))


In [7]:
# Spec validates base_type is actually a type
try:
    invalid_spec = Spec("not_a_type", name="field")
except ValueError as e:
    print(f"✓ Type validation: {e}")

# Generic types work
list_spec = Spec(list[str], name="tags")
dict_spec = Spec(dict[str, int], name="counts")
print(f"\n✓ Generic types accepted: {list_spec.base_type}, {dict_spec.base_type}")

✓ Type validation: base_type must be a type or type annotation, got not_a_type

✓ Generic types accepted: list[str], dict[str, int]


## 3. Metadata Access Patterns

Spec provides multiple ways to access metadata: dict-like `[]`, `.get()`, and properties.

In [8]:
spec = Spec(str, name="username", nullable=True, default="anonymous", description="User login name")

# Dict-like access with []
print(f"Name via []: {spec['name']}")
print(f"Nullable via []: {spec['nullable']}")

# KeyError for missing keys
try:
    missing = spec["nonexistent"]
except KeyError as e:
    print(f"\n✓ KeyError for missing: {e}")

Name via []: username
Nullable via []: True

✓ KeyError for missing: "Metadata key 'nonexistent' undefined in Spec."


In [9]:
# .get() with default value
print(f"Name via .get(): {spec.get('name')}")
print(f"Missing with default: {spec.get('missing', 'default_value')}")

# .get() returns Undefined if key missing and no default
result = spec.get("nonexistent")
print(f"Missing without default: {result}")
print(f"Is Undefined: {result is Undefined}")

Name via .get(): username
Missing with default: default_value
Missing without default: Undefined
Is Undefined: True


In [10]:
# Custom metadata keys work too
custom_spec = Spec(int, name="count", min_value=0, max_value=100, unit="items")

print("Custom metadata:")
print(f"  min_value: {custom_spec['min_value']}")
print(f"  max_value: {custom_spec['max_value']}")
print(f"  unit: {custom_spec['unit']}")

Custom metadata:
  min_value: 0
  max_value: 100
  unit: items


## 4. Spec Properties

Spec provides convenience properties for common metadata: name, is_nullable, is_listable, default.

In [11]:
spec = Spec(str, name="email", nullable=True, listable=True, default="noreply@example.com")

# Property access
print(f"Name property: {spec.name}")
print(f"Is nullable: {spec.is_nullable}")
print(f"Is listable: {spec.is_listable}")
print(f"Default: {spec.default}")

Name property: email
Is nullable: True
Is listable: True
Default: noreply@example.com


In [12]:
# Properties return sensible defaults when metadata missing
minimal_spec = Spec(int)

print(f"Name (undefined): {minimal_spec.name}")
print(f"Is undefined: {minimal_spec.name is Undefined}")
print(f"Is nullable (missing = False): {minimal_spec.is_nullable}")
print(f"Is listable (missing = False): {minimal_spec.is_listable}")
print(f"Default (undefined): {minimal_spec.default}")

Name (undefined): Undefined
Is undefined: True
Is nullable (missing = False): False
Is listable (missing = False): False
Default (undefined): Undefined


In [13]:
# Nullable/listable are boolean properties - explicit check
nullable_spec = Spec(str, nullable=True)
not_nullable_spec = Spec(str, nullable=False)
no_nullable_spec = Spec(str)  # Missing = False

print(f"Explicitly nullable: {nullable_spec.is_nullable}")
print(f"Explicitly not nullable: {not_nullable_spec.is_nullable}")
print(f"Missing (treated as False): {no_nullable_spec.is_nullable}")

Explicitly nullable: True
Explicitly not nullable: False
Missing (treated as False): False


## 5. Default Values and Factories

Spec supports default values (static) and default factories (callable), including async factories.

In [14]:
# Static default value
static_spec = Spec(str, name="status", default="pending")

print(f"Has default: {static_spec.default is not Undefined}")
print(f"Default value: {static_spec.default}")
print(f"Has factory: {static_spec.has_default_factory}")

# Create default value
value = static_spec.create_default_value()
print(f"Created value: {value}")

Has default: True
Default value: pending
Has factory: False
Created value: pending


In [15]:
# Default factory (callable)
import datetime as dt

factory_spec = Spec(dt.datetime, name="created_at", default_factory=lambda: dt.datetime.now(dt.UTC))

print(f"Has factory: {factory_spec.has_default_factory}")
print(f"Is async factory: {factory_spec.has_async_default_factory}")

# Factory called each time
value1 = factory_spec.create_default_value()
import time

time.sleep(0.01)
value2 = factory_spec.create_default_value()

print(f"\nValue 1: {value1}")
print(f"Value 2: {value2}")
print(f"Different instances: {value1 != value2}")

Has factory: True
Is async factory: False

Value 1: 2025-11-23 19:07:18.009767+00:00
Value 2: 2025-11-23 19:07:18.022180+00:00
Different instances: True


In [16]:
import asyncio


# Async default factory
async def async_factory():
    """Simulate async default creation."""

    await asyncio.sleep(0.001)
    return "async_value"


# Warning issued for async factories
import warnings

with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    async_spec = Spec(str, name="async_field", default_factory=async_factory)
    if w:
        print(f"✓ Warning: {w[0].message}")

print(f"\nHas async factory: {async_spec.has_async_default_factory}")

✓ Warning: Async default factories are not yet fully supported by all adapters. Consider using sync factories for compatibility.

Has async factory: True


In [17]:
# Async factory requires acreate_default_value()
try:
    value = async_spec.create_default_value()  # Sync method fails
except ValueError as e:
    print(f"✓ Sync method rejects async factory: {e}")

# Use async method

value = await async_spec.acreate_default_value()
print(f"\nAsync created value: {value}")

# acreate_default_value works for sync factories too
sync_value = await factory_spec.acreate_default_value()
print(f"Sync factory via async method: {sync_value}")

✓ Sync method rejects async factory: Default factory is asynchronous; cannot create default synchronously. Use 'await spec.acreate_default_value()' instead.

Async created value: async_value
Sync factory via async method: 2025-11-23 19:07:18.031819+00:00


In [18]:
# No default - raises error
no_default_spec = Spec(str, name="required")

try:
    value = no_default_spec.create_default_value()
except ValueError as e:
    print(f"✓ No default error: {e}")

✓ No default error: No default value or factory defined in Spec.


## 6. Validators and Validation

Spec supports attaching validators - functions that validate field values.

In [19]:
# Single validator function
def validate_email(value: str) -> str:
    """Validate email format."""
    if "@" not in value:
        raise ValueError(f"Invalid email: {value}")
    return value


email_spec = Spec(str, name="email", validator=validate_email)

print(f"Has validator: {email_spec.get('validator') is not Undefined}")
print(f"Validator: {email_spec['validator']}")

# Validator would be called by Operable during validation
valid_email = email_spec["validator"]("user@example.com")
print(f"\n✓ Valid email: {valid_email}")

try:
    email_spec["validator"]("invalid_email")
except ValueError as e:
    print(f"✓ Validator caught error: {e}")

Has validator: True
Validator: <function validate_email at 0x10734a840>

✓ Valid email: user@example.com
✓ Validator caught error: Invalid email: invalid_email


In [20]:
# Multiple validators (list)
def not_empty(value: str) -> str:
    if not value.strip():
        raise ValueError("Value cannot be empty")
    return value


def min_length(value: str, min_len: int = 3) -> str:
    if len(value) < min_len:
        raise ValueError(f"Value must be at least {min_len} chars")
    return value


multi_validator_spec = Spec(str, name="username", validator=[not_empty, min_length])

validators = multi_validator_spec["validator"]
print(f"Validators: {validators}")
print(f"Count: {len(validators)}")

# Run all validators
value = "alice"
for validator in validators:
    value = validator(value)
print(f"\n✓ All validators passed: {value}")

Validators: [<function not_empty at 0x10734a3e0>, <function min_length at 0x10734a200>]
Count: 2

✓ All validators passed: alice


In [21]:
# Validator validation - must be callable
try:
    invalid = Spec(str, validator="not_a_function")
except ExceptionGroup as eg:
    print(f"✓ Validator validation: {eg.exceptions[0]}")

try:
    invalid = Spec(str, validator=[validate_email, "not_callable"])
except ExceptionGroup as eg:
    print(f"✓ List validator validation: {eg.exceptions[0]}")

✓ Validator validation: Validators must be a list of functions or a function
✓ List validator validation: Validators must be a list of functions or a function


## Summary Checklist

After completing Part 1, you should understand:

- ✅ **CommonMeta**: Standard metadata keys (name, default, nullable, etc.)
- ✅ **Spec Construction**: Creating Spec instances with `base_type` and metadata
- ✅ **Metadata Access**: Using `.name`, `.default`, `.nullable` properties and `get()` method
- ✅ **Properties**: `base_type`, `metadata`, `annotation`, `model_field_type`
- ✅ **Default Values**: Using `default` and `default_factory` for field defaults
- ✅ **Validators**: Single and multiple validators for runtime validation

**Next**: [Part 2 (Advanced)](types_spec_advanced.ipynb) covers transformations, type annotations, Operable integration, and advanced patterns.